<a href="https://colab.research.google.com/github/saulolks/HandTracking/blob/master/hand_gestures.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Entrega 2 - Projeto PDI
### Aplicação de filtros para obtenção do objeto de interesse removendo partes desnecessárias da imagem.

In [0]:
import cv2
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from google.colab.patches import cv2_imshow

In [0]:
wsize_open = 5
wsize_gaussian = 5

y_min = 54
y_max = 137
crmin = 135
crmax = 174
cbmin = 0 
cbmax = 125

In [0]:
headcascade = cv2.CascadeClassifier('face.xml')
backSub = cv2.createBackgroundSubtractorKNN(history=500, dist2Threshold=30)
kernel = np.ones((wsize_open, wsize_open), np.uint8)

In [0]:
original = cv2.imread('sample.jpg')
image = cv2.cvtColor(original, cv2.COLOR_BGR2YCR_CB)

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()
ax[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(image)
ax[1].set_title('YCrCb')
ax[1].set_axis_off()

error: ignored

## Localiza Face
Localiza a face na imagem em tons de cinza.

In [0]:
image_gray = cv2.cvtColor(original, cv2.COLOR_BGR2GRAY)
faces = headcascade.detectMultiScale(image_gray, 1.3, 5)

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()
ax[0].imshow(cv2.cvtColor(original, cv2.COLOR_BGR2RGB))
ax[0].set_title('Original')
ax[0].set_axis_off()

ax[1].imshow(image_gray, cmap='gray')
ax[1].set_title('Gray')
ax[1].set_axis_off()

### Divide imagem YCrCb em 3 canais

In [0]:
y, cr, cb = cv2.split(image)

### Binarização

In [0]:
_, y = cv2.threshold(y, y_min, y_max, cv2.THRESH_BINARY)
_, cr = cv2.threshold(cr, crmin, crmax, cv2.THRESH_BINARY)
_, cb = cv2.threshold(cb, cbmin, cbmax, cv2.THRESH_BINARY)

### Morfologia
Utiliza transformações morfológicas de abertura para remoção de pequenos artefatos na imagem.

In [0]:
y = cv2.morphologyEx(y, cv2.MORPH_OPEN, kernel)
cr = cv2.morphologyEx(cr, cv2.MORPH_OPEN, kernel)
cb = cv2.morphologyEx(cb, cv2.MORPH_OPEN, kernel)

### Merge

In [0]:
def remove_concomponent(img, min_value):
    shape = (len(img), len(img[0]))
    img2 = np.zeros(shape)

    nb_components, output, stats, _ = cv2.connectedComponentsWithStats(img, connectivity=8)
    sizes = stats[1:, -1]
    nb_components = nb_components - 1

    for i in range(0, nb_components):
        if sizes[i] >= min_value:
            img2[output == i + 1] = 255

    return img2.astype(np.uint8)

In [0]:
img = y + cr + cb

_, img = cv2.threshold(img, 100, 255, cv2.THRESH_BINARY)

fig, axes = plt.subplots(1, 2, figsize=(15, 8))
ax = axes.ravel()

ax[0].imshow(img, cmap='gray')
ax[0].set_title('Binary')
ax[0].set_axis_off()

mask = backSub.apply(image)
img = cv2.bitwise_and(mask, img)
img = cv2.medianBlur(img, wsize_gaussian)

ax[1].imshow(img, cmap='gray')
ax[1].set_title('Filtro Média')
ax[1].set_axis_off()

img = remove_concomponent(img=img, min_value=150)


### Removendo Face

In [0]:
marge = 30
for (x, y, w, h) in faces:
    cv2.rectangle(img, (x - marge, y - h + marge), (x + marge + w, y + marge + h), (0, 0, 0), -1)
    
cv2_imshow(img)